### 데이터 수집
    * 스크래이핑을 통한 수집
        - 뉴스
        - SNS
        - OTT
        - 댓글
    * json api를 활용
        - 핀테크 금융
        - 프롭테크 부동산
        - 마케팅
        - SCM
        - CRM
    * 빅데이터 생성
    * DB 활용
    * 시각화
    * 사무 자동화
        - 오피스
        - RPA
    * 사회소통능력 
        - 유튜브
        - SNS
        -홈페이지 관리
### 개별 데이터 분석
    * 정형 데이터 분석: 개별분석
        - 모멘텀 분석
        - 기초 분석
    * 비정형 데이터 분석: 개별분석
        - Count Vectorize
        - TF-IDF
        - Word2Vec:cbow
        - LDA

### 데이터 융합
    * 정형데이터: 주가 데이터
    * 비정형 데이터: 뉴스 데이터
    * 데이터 융합
        - 데이터베이스 융합: DB/SQL join
        - Pandas 활용: Merge 명령어 사용
### 고급 데이터 처리 기법
*  해당분야 용어 기법
* 

In [1]:
### 기초데이터: DB join 방식 활용
### count 벡터화: Pandas 활용
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

In [2]:
dbname='./navernews.db'
def getDB(sql='select nidx,ndate from nnews where nday is NULL'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        qry=cur.execute(sql)
        desc=[row[0] for row in qry.description]
        data=qry.fetchall()
    return((desc,data))

In [3]:
def dbNday(ndays):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        sql='update  nnews set nday="'+ndays[1]+'" where nidx = '+str(ndays[0])
        qry=cur.execute(sql)
    return(qry)

In [4]:
def updateNdays():
    res=getDB()
    ndays=[]
    for r in res[1]:
        ndays.append((r[0],r[1][:10]))
    for n in ndays:
        print(dbNday(n))

In [6]:
updateNdays()

In [8]:
sql= "select sday,sopen,shigh,slow,sclose,svol,count(nidx) nd,group_concat(ntitle) gtitle from stockp a left outer join nnews b on a.sday=b.nday group by sday"
mdata=getDB(sql)

In [9]:
df=pd.DataFrame(data=mdata[1],columns=mdata[0])
df.tail()

,sday,sopen,shigh,slow,sclose,svol,nd,gtitle
885,2021-03-29,282000,282000,276000,277000,53284,0,None
886,2021-03-30,276500,279500,276000,279000,41692,0,None
887,2021-03-31,279000,281500,276500,281000,58150,0,None
888,2021-04-01,282500,283000,277500,279000,72605,0,None
889,2021-04-02,278000,278500,274000,274500,55290,0,None


In [53]:
nlpdf=df[['sday','gtitle']].copy()
nf=nlpdf.fillna('')
nf.head()

,sday,gtitle
0,2017-08-16,
1,2017-08-17,
2,2017-08-18,
3,2017-08-21,
4,2017-08-22,


In [46]:
from konlpy.tag import Kkma
kkma=Kkma()

def dictsort(r):
    sr=sorted(r.items(),key=(lambda x:x[1]),reverse=True)
    return r

def getPOS(rres,limlen=0):
    p=kkma.pos(rres)
    dictpos={}
    words=[]
    #표제어 추출
    check=['NNG','NNP','NNB','NNM','VV','VA','VXV','VXA','VCP','VCN','SF','EFN','EFQ']
    for  w,pos in p:
        if(pos in check):
            if(len(w)>limlen):
                words.append(w)
                if (w in dictpos.keys()):
                    dictpos[w]+=1
                else:
                    dictpos[w]=1
    return (dictpos,words)  

In [60]:
gtitles=nf.values
gtitles[-200:-195]

array([['2020-06-12', ''],
       ['2020-06-15', ''],
       ['2020-06-16', ''],
       ['2020-06-17',
        ' 포토  신세계  푸빌라 캐릭터 상품, 포토  신세계 푸빌라 비치백  비치타올, 포토  신세계백  푸빌라 캐릭터 상품 선보여  , 포토  신세계백화점  푸빌라 캐릭터'],
       ['2020-06-18', '신세계百  휴가철 앞두고 대형  와인 장터  연다']], dtype=object)

In [68]:
corpus=[]
for g in gtitles:
    res=getPOS(g[1],1)
    corpus.append(' '.join(res[1]))

In [119]:

# 카운트 기반 TDM
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(min_df=2)
tdm=vect.fit_transform(corpus).toarray()
tdm.shape

(890, 1998)

In [125]:
## 융합형 자료에서는 단위가 달라서 사전 스케일링이 필요
from sklearn.preprocessing import MinMaxScaler
mxs=MinMaxScaler()
mxtdm=mxs.fit(tdm)
mxtdm=mxs.transform(tdm)
mxtdm.shape

(890, 1998)

In [129]:
### 주피터에서 리스트 축약없이 전체 보기
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
mxtdm[-197]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [103]:
cols=vect.get_feature_names()
cols.insert(0,'date')
cols

['date',
 'ㄴ다',
 'ㄹ까',
 'ㄹ래',
 'ㅂ니다',
 '가격',
 '가공',
 '가구',
 '가나',
 '가능',
 '가능성',
 '가닥',
 '가도',
 '가동',
 '가드',
 '가란',
 '가르',
 '가방',
 '가성',
 '가세',
 '가속',
 '가속화',
 '가시화',
 '가을',
 '가입',
 '가입금',
 '가전',
 '가정',
 '가족',
 '가지',
 '가짜',
 '가치',
 '가칭',
 '각축전',
 '간식',
 '간판',
 '간편',
 '갈리',
 '갈이',
 '감각',
 '감독',
 '감면',
 '감성',
 '감소',
 '감안',
 '감자',
 '강남',
 '강등',
 '강릉',
 '강세',
 '강원',
 '강원도',
 '강의',
 '강자',
 '강정',
 '강좌',
 '강화',
 '갖추',
 '개관',
 '개념',
 '개막',
 '개막전',
 '개명',
 '개발',
 '개봉',
 '개선',
 '개설',
 '개시',
 '개원',
 '개월',
 '개장',
 '개점',
 '개최',
 '개편',
 '객실',
 '갤러리',
 '갱신',
 '거래액',
 '거론',
 '거리',
 '거침없',
 '거포',
 '걱정',
 '건강',
 '건립',
 '건물',
 '건설',
 '걸음',
 '검사',
 '검토',
 '게시판',
 '게임',
 '겨냥',
 '겨울',
 '격돌',
 '격변기',
 '견인',
 '결과',
 '결국',
 '결단',
 '결별',
 '결산',
 '결실',
 '결정',
 '결제',
 '결합',
 '경기',
 '경기장',
 '경복궁',
 '경북',
 '경영',
 '경영난',
 '경쟁',
 '경쟁력',
 '경제',
 '경험',
 '계단',
 '계속',
 '계승',
 '계약',
 '계열',
 '계열사',
 '계절',
 '계층',
 '계획',
 '고가',
 '고객',
 '고공',
 '고급',
 '고기',
 '고난',
 '고려',
 '고르',
 '고민',
 '고성',
 '고속',
 '고용',
 '고전',
 '고창',
 '고향',


In [79]:
gtitles.shape

(890, 2)

In [70]:
tdm[-200:-195]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [130]:
tdms=[]
for i,t in enumerate(mxtdm):
    res=list(t)
    res.insert(0,gtitles[i][0])
    tdms.append(res)

In [131]:
tdmdf=pd.DataFrame(data=tdms, columns=cols)
stdmdf=tdmdf.set_index('date')
stdmdf.head()

,ㄴ다,ㄹ까,ㄹ래,ㅂ니다,가격,가공,가구,가나,가능,가능성,...,흐름,흑자,흔들,흥행,희망,희비,희석,히트,힘들,힘주
date,,,,,,,,,,,,,,,,,,,,,
2017-08-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
sdf=df.set_index('sday').drop(['gtitle'],axis=1)
mxs=MinMaxScaler()
mxs.fit(sdf)
outv=mxs.transform(sdf)
msdf=pd.DataFrame(outv,index=sdf.index,columns=sdf.columns)
msdf.head()

,sopen,shigh,slow,sclose,svol,nd
sday,,,,,,
2017-08-16,0.057823,0.063248,0.074906,0.069930,0.101734,0.0
2017-08-17,0.061224,0.058120,0.069288,0.066434,0.057753,0.0
2017-08-18,0.054422,0.063248,0.071161,0.064685,0.053122,0.0
2017-08-21,0.061224,0.058120,0.052434,0.047203,0.084243,0.0
2017-08-22,0.042517,0.046154,0.059925,0.059441,0.037539,0.0


In [141]:
### 비정형데이터와 정형 데이터의 융합
mergedf=pd.merge(msdf,stdmdf,left_index=True,right_index=True)
mergedf.head()

,sopen,shigh,slow,sclose,svol,nd,ㄴ다,ㄹ까,ㄹ래,ㅂ니다,...,흐름,흑자,흔들,흥행,희망,희비,희석,히트,힘들,힘주
sday,,,,,,,,,,,,,,,,,,,,,
2017-08-16,0.057823,0.063248,0.074906,0.069930,0.101734,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-17,0.061224,0.058120,0.069288,0.066434,0.057753,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-18,0.054422,0.063248,0.071161,0.064685,0.053122,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-21,0.061224,0.058120,0.052434,0.047203,0.084243,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-08-22,0.042517,0.046154,0.059925,0.059441,0.037539,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
mergedf['target']=mergedf['sclose'].shift(-1)
mergedf.head()

,sopen,shigh,slow,sclose,svol,nd,ㄴ다,ㄹ까,ㄹ래,ㅂ니다,...,흑자,흔들,흥행,희망,희비,희석,히트,힘들,힘주,target
sday,,,,,,,,,,,,,,,,,,,,,
2017-08-16,0.057823,0.063248,0.074906,0.069930,0.101734,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066434
2017-08-17,0.061224,0.058120,0.069288,0.066434,0.057753,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064685
2017-08-18,0.054422,0.063248,0.071161,0.064685,0.053122,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047203
2017-08-21,0.061224,0.058120,0.052434,0.047203,0.084243,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.059441
2017-08-22,0.042517,0.046154,0.059925,0.059441,0.037539,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052448


In [148]:
Xori=mergedf.iloc[:,:2004].values
Xori.shape

(890, 2004)

In [151]:
Yori=mergedf.fillna(0).iloc[:,2004].values
Yori

array([0.06643357, 0.06468531, 0.0472028 , 0.05944056, 0.05244755,
       0.05594406, 0.0541958 , 0.06118881, 0.0541958 , 0.0506993 ,
       0.0472028 , 0.02797203, 0.02447552, 0.02622378, 0.00874126,
       0.01223776, 0.0034965 , 0.02272727, 0.01923077, 0.01048951,
       0.00174825, 0.02972028, 0.02972028, 0.02972028, 0.01573427,
       0.00874126, 0.00874126, 0.        , 0.03146853, 0.01398601,
       0.04020979, 0.0541958 , 0.04370629, 0.02622378, 0.02972028,
       0.06643357, 0.11363636, 0.10839161, 0.1013986 , 0.10664336,
       0.11013986, 0.12937063, 0.12062937, 0.12237762, 0.16083916,
       0.15909091, 0.18706294, 0.17307692, 0.1486014 , 0.15734266,
       0.16958042, 0.16783217, 0.23426573, 0.28146853, 0.25699301,
       0.24475524, 0.23251748, 0.25524476, 0.25      , 0.24825175,
       0.27797203, 0.33566434, 0.31293706, 0.32167832, 0.29020979,
       0.29370629, 0.33741259, 0.34265734, 0.34265734, 0.37587413,
       0.36713287, 0.38636364, 0.4020979 , 0.37587413, 0.35489

In [160]:
### 윈도우 처리
tot=len(Xori)
winsize=5
X_data=[]
Y_data=[]
for i in range(tot-winsize):
    X_data.append(Xori[i:winsize+i])
    Y_data.append(Xori[winsize+i][3])
    #Y_data.append(Yori[winsize-1+i])
x_data=numpy.array(X_data).astype('float32')
y_data=numpy.array(Y_data).astype('float32')

In [140]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [158]:
model=Sequential()
model.add(LSTM(128,input_shape=(winsize,2004,)))
model.add(Dense(64,activation ="linear"))
model.add(Dense(1,activation="linear"))
model.compile(optimizer="sgd",loss="mse",metrics=['cosine_proximity'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               1092096   
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,100,417
Trainable params: 1,100,417
Non-trainable params: 0
_________________________________________________________________


In [161]:
model.fit(x_data,y_data,epochs=10)

Epoch 1/10
28/28 [==============================] - 4s 25ms/step - loss: 0.0540 - cosine_proximity: 0.9677
Epoch 2/10
28/28 [==============================] - 1s 24ms/step - loss: 0.0164 - cosine_proximity: 0.9973
Epoch 3/10
28/28 [==============================] - 1s 24ms/step - loss: 0.0119 - cosine_proximity: 0.9999
Epoch 4/10
28/28 [==============================] - 1s 26ms/step - loss: 0.0100 - cosine_proximity: 0.9996
Epoch 5/10
28/28 [==============================] - 1s 27ms/step - loss: 0.0076 - cosine_proximity: 0.9997
Epoch 6/10
28/28 [==============================] - 1s 26ms/step - loss: 0.0064 - cosine_proximity: 0.9993
Epoch 7/10
28/28 [==============================] - 1s 26ms/step - loss: 0.0049 - cosine_proximity: 0.9987
Epoch 8/10
28/28 [==============================] - 1s 29ms/step - loss: 0.0050 - cosine_proximity: 0.9994
Epoch 9/10
28/28 [==============================] - 1s 28ms/step - loss: 0.0039 - cosine_proximity: 0.9999 0s - loss: 0.0040 - cosine_proxi
Epoc